In [ ]:
import scipy.sparse.linalg as LA
from scipy import sparse
import matplotlib.pylab as plt
import os
import py_plantbox as pb
from rb_tools import *

import xylem_flux 

import timeit

import numpy as np
from scipy.integrate import odeint
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
from vtk import *
from IPython.display import Image
from vtk.util import numpy_support as VN
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def get_organ_color(i):
    if i == 2:
        return "wheat"
    elif i == 4:
        return "#5b965b"
    elif i == 8:
        return "lightgreen"
    else:
        return "black";


all_5_plant_name = ["morning_glory_2.5m_d", "morning_glory_3.5m_d","morning_glory_9m_d","morning_glory_10m_d","morning_glory_14m_d"]

# Simulate a root system

# name = "morning_glory_7m"
# name = "morning_glory_2.5m_d"
# name = "morning_glory_3.5m_d"
# name = "morning_glory_9m_d"
# name = "morning_glory_10m_d"
#name = "morning_glory_14m_d"
#name = "Crypsis_aculeata_Clausnitzer_1994"
#name = "Triticum_aestivum_a_Bingham_2011"
#name = "sympodial_dichasium"
#name = "sympodial_monochasium"
# name = "monopodial"
i=0
for i in range(0,5):
    name = all_5_plant_name[i]
    plant = pb.Plant()
    #sca = pb.Organ.getScalar('organtype')
    plant.openXML(name)
    plant.initialize()
    ana = pb.SegmentAnalyser(plant)

    plant.simulate(180)
    #for i in range (0,180):
    #    plant.simulate(i)
    # for i in range(0,7):
    #     plant.simulate(1) 



    #seg = np.vstack((seg,lseg))
    # print("number of root segments",len(rseg))
    # print("number of shoot segments",len(sseg))
    # print("number of segments",len(seg))

    #rootnodes = v2a(sca('organtype'))

    #print(rseg)
    #print(sseg)
    #print(lseg)
    #print(sseg[1])

    #print(len(nodes))
    #print(len(rseg))
    #print(len(lseg))



    plant.write("../results/"+name+".vtp",15)
plt.rcParams['figure.figsize'] = [30, 10]
plt.rcParams.update({'font.size': 16})


In [ ]:

def plot_vtp(name, indentx, indentz,scale , title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/100 +0.4, indentz/50+0.6, title,rotation=90 , transform=ax.transAxes,size=20 )
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ ((n)*scale+indentx)*4.8 for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=10)

i=0
plt.rcParams['figure.figsize'] = [20, 40]
fig = plt.figure("Stem Branching and Leaf Arrangements")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(0, 270)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

a = Arrow3D([-0.6,-0.6],[0,0],[0,2.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(a)
ax.text(-2, 0 , 1.9, "2.5 m", (0,0,1), color='k', fontsize=58 )


b = Arrow3D([1.9,1.9],[0,0],[0,3.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(b)
ax.text(0.7, 0 , 2.6, "3.5 m", (0,0,1), color='k', fontsize=58 )

c = Arrow3D([4.25,4.25],[0,0],[0,9], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(c)
ax.text(3.2, 0 , 4.4, "9 m", (0,0,1), color='k', fontsize=58 )

d = Arrow3D([6.55,6.55],[0,0],[0,10], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(d)
ax.text(5.5, 0 , 7, "10 m", (0,0,1), color='k', fontsize=58 )

e = Arrow3D([8.85,8.85],[0,0],[0,14], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(e)
ax.text(7.8, 0 , 10, "14 m", (0,0,1), color='k', fontsize=58 )













#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)
for i in range(0,5):
    plot_vtp(all_5_plant_name[i],0,0,1," ")

In [ ]:
from matplotlib.patches import Circle, PathPatch
import mpl_toolkits.mplot3d.art3d as art3d
from matplotlib.transforms import Affine2D
def plot_vtp(name, indentx, indentz,scale , title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/100 +0.4, indentz/50+0.6, title,rotation=90 , transform=ax.transAxes,size=20 )
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ ((n)*scale+indentx) for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=30)
plt.rcParams['figure.figsize'] = [10, 40]
fig = plt.figure("pressure")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(0, 90)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')
p = Circle((0, 4), 3.5,fill=False, linewidth=10, alpha=0.5)
#ax.add_patch(p)
art3d.pathpatch_2d_to_3d(p, z=0, zdir="x")

a = Arrow3D([-0.02,-0.02],[0,0],[2.4,1.9], mutation_scale=40, lw=5, arrowstyle="-|>", color="tomato", label="")
#ax.add_artist(a)

b = Arrow3D([0.14,0.14],[0,0],[1.3,7.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="#FF6347", label="")
ax.add_artist(b)

c = Arrow3D([-0.12,-0.02],[2.3,2.3],[5,5], mutation_scale=20, lw=5, arrowstyle="->", color="k", label="")
#ax.add_artist(c)
ax.text(-0.03, 0 ,4.4, "\u2190\n Loading", (1,0,0), color='#FF6347', fontsize=50,
        horizontalalignment='left' )
ax.text(0.19, 0 , 5.5, "1-Source $\Delta P$ = 0.49 MPa", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
b = Arrow3D([0.05,0.05],[0,0],[0,7.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(b)
ax.text(0.14, 0 , 0.6, "7.5 m", (0,0,0), color='k', fontsize=50,
        horizontalalignment='center' )
ax.text(0.08, 0 , 1.3, "\u25CF", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
ax.text(0.08, 0 , 2.3, "\u25CF", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
ax.text(0.08, 0 , 3.3, "\u25CF", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
ax.text(0.08, 0 , 5.3, "\u25CF", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
ax.text(-0.05, 0 , 6.35, "\u25CFPressure \nmeasured", (0,0,0), color='tomato', fontsize=50,
        horizontalalignment='left', verticalalignment='bottom' )


ax.text(-0.09, 0 , 2.0, "\u2193Flow\n       Measured", (1,0,0), color='tomato', fontsize=50,
        horizontalalignment='center' )


ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])

plot_vtp("morning_glory_7m",0,0,1," ")

In [ ]:
i=0
plt.rcParams['figure.figsize'] = [20, 40]
fig = plt.figure("Stem Branching and Leaf Arrangements")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(0, 270)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

a = Arrow3D([-0.6,-0.6],[0,0],[0,2.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(a)
ax.text(-2, 0 , 1.9, "2.5 m", (0,0,1), color='k', fontsize=58 )


b = Arrow3D([1.9,1.9],[0,0],[0,3.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(b)
ax.text(0.7, 0 , 2.6, "3.5 m", (0,0,1), color='k', fontsize=58 )

c = Arrow3D([4.25,4.25],[0,0],[0,9], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(c)
ax.text(3.2, 0 , 4.4, "9 m", (0,0,1), color='k', fontsize=58 )

d = Arrow3D([6.55,6.55],[0,0],[0,10], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(d)
ax.text(5.5, 0 , 7, "10 m", (0,0,1), color='k', fontsize=58 )

e = Arrow3D([8.85,8.85],[0,0],[0,14], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(e)
ax.text(7.8, 0 , 10, "14 m", (0,0,1), color='k', fontsize=58 )


f = Arrow3D([-4,-4],[0,0],[0,7.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="k", label="")
ax.add_artist(f)
ax.text(-5, 0 , 1.9, "7.5 m", (0,0,1), color='k', fontsize=58 )



b = Arrow3D([0.14,0.14],[0,0],[1.3,7.5], mutation_scale=20, lw=5, arrowstyle="|-|", color="tomato", label="")
#ax.add_artist(b)

#c = Arrow3D([-0.12,-0.02],[2.3,2.3],[5,5], mutation_scale=20, lw=5, arrowstyle="->", color="k", label="")
#ax.add_artist(c)
ax.text(-2.08, 0 , 12, " Equal\n Loading\n Speed", (1,0,0), color='k', fontsize=50,
        horizontalalignment='left' )
ax.text(-3, 0 , 12, "\u2190 Calibration", (0,0,1), color='tomato', fontsize=50,
        horizontalalignment='center' )
#ax.text(-0.14, 0 , 2.0, "\u2193 Flow Direction", (1,0,0), color='royalblue', fontsize=50,
#        horizontalalignment='center' )









#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)
for i in range(0,5):
    plot_vtp(all_5_plant_name[i],0,0,1," ")
plot_vtp("morning_glory_7m",-3,0,1," ")

In [ ]:
h_font = {'fontname':'DejaVu Sans Mono'}
def plot_vtp(name, indentx, indentz,scale , title, oren):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/13 +0.3, indentz/23+0.57, title,rotation=0 , transform=ax.transAxes, size=20,**h_font)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ (n)*scale+indentx-1 for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=2)


   

#name = "sympodial_dichasium"
plt.rcParams['figure.figsize'] = [20, 40]
fig = plt.figure("Stem Branching and Leaf Arrangements")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(45, 270)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)
plot_vtp("monopodial",0,0,1,"Monopodial",1)

plot_vtp("sympodial_dichasium",2,0,1," Sympodial \n dichasium",1)
plot_vtp("sympodial_monochasium",4,0,1," Sympodial \n monochasium",0.5)


plot_vtp("leaf_opposite_decussate",0,-5,50, " Opposite\n decussate",1)
plot_vtp("leaf_alternate",2,-5,50, "Alternate",1)
plot_vtp("leaf_spiral",4,-5,50, "Spiral" ,-1)
ax.text2D(0/13 +0.3, 3/23+0.57, "A",rotation=0 , transform=ax.transAxes, size=30, **h_font)
ax.text2D(0/13 +0.3, -1.9/23+0.57, "B",rotation=0 , transform=ax.transAxes, size=30, **h_font)

ax.text2D(6/13 +0.3, 2.2/23+0.57, "Branching Patterns",rotation=90 , transform=ax.transAxes, size=30, **h_font)
ax.text2D(6/13 +0.3, -2.2/23+0.57, "Leaf Arrangements",rotation=90 , transform=ax.transAxes, size=30, **h_font)


In [ ]:
def plot_vtp(name, indentx, indentz,scale , title, oren):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/13 +0.3, indentz/23+0.57, title,rotation=90 , transform=ax.transAxes, size=20)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ (n)*scale+indentx-1 for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=2)
        if vtk_organtype[i]==8:
            x = [ (n)*scale+indentx-1 for n in np_pts[:,0]/100]
        
            z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

            y = [ n*scale for n in np_pts[:,1]/100]
            add_leaf(x[0],y[0],z[0])

   

#name = "sympodial_dichasium"
plt.rcParams['figure.figsize'] = [20, 40]
fig = plt.figure("Stem Branching and Leaf Arrangements")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(45, 270)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)

plot_vtp("leaf_spiral",4,-5,50, "Spiral" ,-1)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.tri as mtri
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

add_leaf(0,0,0)

plt.show()

In [ ]:
def add_leaf(x1,y1,z1,x2,y2,z2):
    
    (x1+x2)/2
    (y1+y2)/2
    (z1+z2)/2
    #Xc = r * np.sin(theta)
    #Yc = r * np.cos(theta)
    xc=np.linspace(x1, x2, 10)
    #xc=np.array([0+x,4+x,4+x,6+x,8+x])
    yc=np.linspace(y1, y2, 10)
    #yc=np.array([0+y,6+y,4+y,4+y,8+y])
    #Xc, Yc=np.meshgrid(x, y)
    triang = mtri.Triangulation(xc, yc)
    #Zc = np.sqrt(1-Xc**3)
    #Zc = np.zeros(5)
    #Zc =f(Xc,Yc,r)
    Zc = np.array([z,z,z,z,z])
    # Drawparameters
    rstride = 1
    cstride = 1
    #ax.plot_surface(Xc, Yc*0.5, Zc, alpha=1, rstride=rstride, cstride=cstride)
    ax.plot_trisurf(triang,Zc)
    ax.view_init(45, 90)
    #ax.plot_surface(Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride)

In [ ]:

def plot_vtp(name, indentx, indentz,scale , title, oren):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/100 +0.4, indentz/50+0.6, title,rotation=90 , transform=ax.transAxes,size=20 )
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ (n)*scale+indentx-1 for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=2)


   


In [ ]:
from rb_tools import *
nodes = vv2a(plant.getNodes())/100 # convert from cm to m 
rseg = seg2a(plant.getSegments(15)) # root system segments
sseg = seg2a(plant.getSegments(4)) # strm system segments
lseg = v2ai(plant.getNodesOrganType())
plant_ana = pb.SegmentAnalyser(plant) 
nodes = vv2a(plant.getNodes())/100 # convert from cm to m 
node_connection_o = seg2a(plant.getSegments(15)) # plant segments
len(nodes)



In [ ]:
# points = np.array( reader.GetOutput().GetPointData() )

# plt.plot(points)
# plt.show()

# print(reader.GetOutput().GetCellData().GetArray("organtype"))








plt.rcParams['figure.figsize'] = [5, 10]
plt.rcParams.update({'font.size': 16})
fig = plt.figure()


# Get rid of colored axes planes
# First remove fill


# Bonus: To get rid of the grid as well:


for i in range(0,5):
    
    plot_vtp(all_5_plant_name[i],i+1)
    

 #ax.dist = 5.0



root_patch = mpatches.Patch(color='orange', label='root')
seed_patch = mpatches.Patch(color='black', label='seed')
stem_patch = mpatches.Patch(color='darkgreen', label='stem')
leaf_patch = mpatches.Patch(color='lightgreen', label='leaf')
# plt.legend(handles=[root_patch, seed_patch,stem_patch,leaf_patch],bbox_to_anchor=(0.7, 0.7), loc=2, borderaxespad=0.)

plt.show() 
fig.savefig("{}.png".format('organtype_structure'), bbox_inches='tight',dpi=300)

In [ ]:
np_pts[:,2]
#    print(pts)
# print(vtk_line)
# len(vtk_line)
# len(vtk_cor)
# len(vtk_organtype)
# vtk_cor[0]
# vtk_cor[43]
plt.rcParams['figure.figsize'] = [5, 10]
plt.rcParams.update({'font.size': 16})
fig = plt.figure()
plot_vtp("morning_glory_14m_dt",1)

# i = 0
# j = vtk_line[0]
# k = 0
# for i in rang(0,len(vtk_line)):
#     if j-1 >0:
        
#         j=vtk_line[i]
#     for i_

In [ ]:
plt.rcParams['figure.figsize'] = [30, 30]
def plot_vtp(name, i_input, title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax = fig.add_subplot(int("23{}".format(i_input)), projection='3d')
    #plt.axis('image')
    plt.axis('equal')
    plt.title(title)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.view_init(0, 90)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_zlabel('')

    ax.set_ylim3d(0,2)
    ax.set_zlim3d(-1.5,0.5)
    ax.set_yticklabels([])
#    ax.set_xticklabels([])
#     ax.set_zticklabels([])
    ax.grid(False)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()    
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        ax.plot3D(np_pts[:,0]/100,np_pts[:,1]/100,np_pts[:,2]/100, c= get_organ_color(vtk_organtype[i]) ,linewidth=1 )

fig = plt.figure()

fig.subplots_adjust(hspace=0, wspace=-0.2)



plot_vtp("Anagallis_femina_Leitner_2010",1,"$Anagallis$ $femina$" )
plot_vtp("Brassica_napus_a_Leitner_2010",2, "$Brassica$ $napus$")
plot_vtp("Brassica_oleracea_Vansteenkiste_2014",3, "$Brassica$ $oleracea$")
plot_vtp("Crypsis_aculeata_Clausnitzer_1994",4, "$Crypsis$ $aculeata$")
plot_vtp("Heliantus_Pagès_2013",5, "$Heliantus$")
plot_vtp("Juncus_squarrosus_Clausnitzer_1994",6, "$Juncus$ $squarrosus$")

In [ ]:

def plot_vtp(name, indent, title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text(indent, 0, -2,  s=title, fontsize=15)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ x+indent for x in np_pts[:,0]/100]
        ax.plot3D(x,np_pts[:,1]/100,np_pts[:,2]/100, c= get_organ_color(vtk_organtype[i]) ,linewidth=1 )

In [ ]:

def plot_vtp(name, indentx, indentz,scale , title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text(indentx+0.5, 0, indentz*2+1,  s=title, fontsize=36, **h_font)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ (n)*scale+indentx for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=2)


   


   



#name = "sympodial_dichasium"
plt.rcParams['figure.figsize'] = [30, 40]
fig = plt.figure("Plants")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
plt.axis('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(45, 90)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)



plot_vtp("Anagallis_femina_Leitner_2010",0,0,4," $Anagallis$\n $femina$" )
plot_vtp("Brassica_napus_a_Leitner_2010",1.5,0,1, "  $Brassica$\n  $napus$")
plot_vtp("Brassica_oleracea_Vansteenkiste_2014",3,0,2, " $Brassica$\n $oleracea$")




plot_vtp("Crypsis_aculeata_Clausnitzer_1994",0,-3,2, " $Crypsis$\n $aculeata$")
plot_vtp("Heliantus_Pagès_2013",1.5,-3,2, " $Heliantus$")
plot_vtp("Juncus_squarrosus_Clausnitzer_1994",3,-3,2, " $Juncus$\n $squarrosus$")
plt.show() 

In [ ]:
import scipy.sparse.linalg as LA
from scipy import sparse
import matplotlib.pylab as plt
import os
import py_plantbox as pb
from rb_tools import *

import xylem_flux 

import timeit

import numpy as np
from scipy.integrate import odeint
#name = "Brassica_oleracea_Vansteenkiste_2014"
#name = "Brassica_oleracea_Vansteenkiste_2014"
#name = "Crypsis_aculeata_Clausnitzer_1994"
#name = "Heliantus_Pagès_2013"
#name = "Juncus_squarrosus_Clausnitzer_1994"

def plot_vtp(name, i_input):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax = fig.add_subplot(int("23{}".format(i_input)), projection='3d')
    plt.axis('image')
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.view_init(0, 90)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_zlabel('')

    ax.set_ylim3d(0,2)
    ax.set_zlim3d(-1.5,0.5)
    ax.set_yticklabels([])
#    ax.set_xticklabels([])
#     ax.set_zticklabels([])
    ax.grid(False)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()    
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        ax.plot3D(np_pts[:,0]/100,np_pts[:,1]/100,np_pts[:,2]/100, c= get_organ_color(vtk_organtype[i]) ,linewidth=1 )



name = "Lupinus_albus_Leitner_2014"

plant = pb.Plant()
#sca = pb.Organ.getScalar('organtype')
plant.openXML(name)
plant.initialize()
ana = pb.SegmentAnalyser(plant)

plant.simulate(65)
#for i in range (0,180):
#    plant.simulate(i)
# for i in range(0,7):
#     plant.simulate(1) 



#seg = np.vstack((seg,lseg))
# print("number of root segments",len(rseg))
# print("number of shoot segments",len(sseg))
# print("number of segments",len(seg))

#rootnodes = v2a(sca('organtype'))

#print(rseg)
#print(sseg)
#print(lseg)
#print(sseg[1])

#print(len(nodes))
#print(len(rseg))
#print(len(lseg))



plant.write("../results/"+name+".vtp",15)

In [ ]:
plt.rcParams['figure.figsize'] = [50, 10]
fig = plt.figure()
plot_vtp("Triticum_aestivum_a_Bingham_2011",5, "$Triticum$ $aestivum$")

In [ ]:
plt.rcParams['figure.figsize'] = [50, 10]
fig = plt.figure()
plot_vtp("Triticum_aestivum",5)

In [ ]:
import scipy.sparse.linalg as LA
from scipy import sparse
import matplotlib.pylab as plt
import os
import py_plantbox as pb
from rb_tools import *

import xylem_flux 

import timeit

import numpy as np
from scipy.integrate import odeint
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
from vtk import *
from IPython.display import Image
from vtk.util import numpy_support as VN
def get_organ_color(i):
    if i == 2:
        return "wheat"
    elif i == 4:
        return "darkgreen"
    elif i == 8:
        return "lightgreen"
    else:
        return "black";

def plot_vtp(name, i_input, title):
    reader = vtk.vtkXMLPolyDataReader()
    path = "../results/{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organtype"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax = fig.add_subplot(int("23{}".format(i_input)), projection='3d')
    plt.axis('equal')
    plt.title(title)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.view_init(45, 90)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_zlabel('')

 #   ax.set_ylim3d(0,2)
#    ax.set_zlim3d(-1.5,0.5)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
#     ax.set_zticklabels([])
    ax.grid(False)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()    
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        ax.plot3D(np_pts[:,0]/100,np_pts[:,1]/100,np_pts[:,2]/100, c= get_organ_color(vtk_organtype[i]) ,linewidth=5 )



all_5_plant_name = ["morning_glory_2.5m_d", "morning_glory_3.5m_d","morning_glory_9m_d","morning_glory_10m_d","morning_glory_14m_d"]

# Simulate a root system

# name = "morning_glory_7m"
# name = "morning_glory_2.5m_d"
# name = "morning_glory_3.5m_d"
# name = "morning_glory_9m_d"
# name = "morning_glory_10m_d"
# name = "morning_glory_14m_d"
#name = "leaf_opposite_decussate"
#name = "leaf_spiral"
name = "leaf_alternate"
plant = pb.Plant()
#sca = pb.Organ.getScalar('organtype')
plant.openXML(name)
plant.initialize()
ana = pb.SegmentAnalyser(plant)

plant.simulate(180)
#for i in range (0,180):
#    plant.simulate(i)
# for i in range(0,7):
#     plant.simulate(1) 



#seg = np.vstack((seg,lseg))
# print("number of root segments",len(rseg))
# print("number of shoot segments",len(sseg))
# print("number of segments",len(seg))

#rootnodes = v2a(sca('organtype'))

#print(rseg)
#print(sseg)
#print(lseg)
#print(sseg[1])

#print(len(nodes))
#print(len(rseg))
#print(len(lseg))



plant.write("../results/"+name+".vtp",15)
plt.rcParams['figure.figsize'] = [30, 30]
plt.rcParams.update({'font.size': 30})
fig = plt.figure()
plot_vtp("leaf_opposite_decussate",1, "Opposite decussate")
plot_vtp("leaf_spiral",2, "Spiral")
plot_vtp("leaf_alternate",3, "Alternate" )

In [ ]:
#name = "sympodial_dichasium"
fig = plt.figure()
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=-0.9)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')
ax.set_aspect('equal')
plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(0, 90)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)
plot_vtp("leaf_opposite_decussate",1, "$\u2191$ Opposite decussate")
plot_vtp("leaf_alternate",2, "$\u2191$ Spiral")
plot_vtp("leaf_tworowed",3, "$\u2191$ Alternate" )

In [24]:
import py_plantbox as pb
import xml.etree.ElementTree as ET
from vtk import *

plant = pb.Plant()
names = ["Juncus_squarrosus_Clausnitzer_1994", "Anagallis_femina_Leitner_2010", 
         "Brassica_napus_a_Leitner_2010", "Brassica_oleracea_Vansteenkiste_2014",
        "CPlantBox_test_leaf_tree", "Crypsis_aculeata_Clausnitzer_1994","Heliantus_Pagès_2013", 
        "Triticum_aestivum_a_Bingham_2011"]
i=0
for i in range(len(names)):
    tree = ET.parse("modelparameter/"+names[i]+".xml")
    dist = 50
    root = tree.getroot()
    seedx=root.find(".//*[@name='seedPos.x']")
    seedy=root.find(".//*[@name='seedPos.y']")
    seedx.set('value',str(dist*(i+1)))
    #seedy.set('value',str(dist*j))
    a=root.find(".//*[@name='seedPos.x']").attrib.get('value')
    
    tree.write('modelparameter/'+str(i)+'.xml')


In [ ]:
import scipy.sparse.linalg as LA
from scipy import sparse
import matplotlib.pylab as plt
import os
import py_plantbox as pb
from rb_tools import *

import xylem_flux 

import timeit

import numpy as np
from scipy.integrate import odeint
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
from vtk import *
from IPython.display import Image
from vtk.util import numpy_support as VN
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
i=0
for i in range(0,9):
    
    plant = pb.Plant()
    #sca = pb.Organ.getScalar('organtype')
    plant.openXML(str(i))
    plant.initialize()
    

    plant.simulate(56)
    ana = pb.SegmentAnalyser(plant)
    ana.write("../results/"+str(i)+".vtp")

In [5]:
from vtk import *
from vtk.util import numpy_support as VN
from CPlantBox_PiafMunch import *
appendFilter = vtk.vtkAppendPolyData()
for i in range(0,10):

    vtpname = '../../results/'+str(i)+'.vtp'
    reader1 = vtk.vtkXMLPolyDataReader()
    input1 = vtk.vtkPolyData()
    path1 = "{}".format(vtpname) #path or name of the vtp output
    reader1.SetFileName(path1)
    reader1.Update()
    input1.ShallowCopy(reader1.GetOutput())
    appendFilter.AddInputData(input1)
    appendFilter.Update()

    #  Remove any duplicate points.
    #cleanFilter = vtk.vtkCleanPolyData()
    #cleanFilter.SetInputConnection(appendFilter.GetOutputPort())
    #cleanFilter.Update()
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName("../../results/all_flat_plant.vtp")
writer.SetInputData(appendFilter.GetOutput()) 
writer.Write()  

1

In [2]:
!rm modelparameter
!ln -s /mnt/c/CPlantBox/modelparameter/ modelparameter

In [4]:
import py_plantbox as pb
from CPlantBox_PiafMunch import *
os.chdir("/mnt/c/CPlantBox/tutorial/jupyter")

for i in range(9,10):
    
    plant1 = CPlantBox_analysis('{}'.format(str(i)), 180, '{}'.format(str(i)))


In [7]:
plant1 = CPlantBox_analysis('{}'.format('0'), 56, '{}'.format(str(i)))